In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from yass.batch import BatchProcessor
from yass.preprocess.filter import _butterworth
from functools import partial
import numpy as np
from os import path
import matplotlib.pyplot as plt
import logging

/Users/Edu/miniconda3/envs/yass/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
logging.basicConfig(level=logging.DEBUG)
CHANNELS = 49

In [3]:
filtering = partial(_butterworth, low_frequency=300,
                    high_factor=0.1, order=3,
                    sampling_frequency=20000)
filtering.__name__ = 'filtering'

In [4]:
path_to_data = path.expanduser('~/data/yass/ej49_data1_set1.bin')
path_to_output = path.expanduser('~/Desktop/out.bin')
data = np.fromfile(path_to_data,
                   dtype='int16').reshape(-1, CHANNELS)

In [11]:
bp = BatchProcessor(path_to_data, dtype='int16', n_channels=CHANNELS,
                    data_order='samples', loader='memmap',
                    max_memory='50MB')

DEBUG:yass.batch.generator:Max memory: 50 MB. Itemsize: 2 bytes


In [12]:
%%timeit -n1 -r1
res = bp.multi_channel_apply(filtering, mode='disk',
                             output_path=path_to_output,
                             processes=1)

INFO:yass.batch.batch:Applying function filtering...
DEBUG:yass.batch.generator:Observations per channel: 6,000,000. Number of channels: 49. Total observations: 294,000,000 Size to traverse: 560.76 MB
DEBUG:yass.batch.generator:Max observations per batch: 26,214,363 (50 MB), 534,987 max observations per channel
DEBUG:yass.batch.generator:Number of batches: 12
DEBUG:yass.batch.generator:Last batch with 5,642,007 observations (11284014), 115,143 observations per channel
DEBUG:yass.batch.util:Processing batch 0...
DEBUG:yass.batch.util:Applying function in batch 0...
DEBUG:yass.batch.util:Done Applying function in batch 0...
DEBUG:yass.batch.util:Processing batch 1...
DEBUG:yass.batch.util:Applying function in batch 1...
DEBUG:yass.batch.util:Done Applying function in batch 1...
DEBUG:yass.batch.util:Processing batch 2...
DEBUG:yass.batch.util:Applying function in batch 2...
DEBUG:yass.batch.util:Done Applying function in batch 2...
DEBUG:yass.batch.util:Processing batch 3...
DEBUG:yass.b

22.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%%timeit -n1 -r1
res = bp.multi_channel_apply(filtering, mode='disk',
                             output_path=path_to_output,
                             processes=6)

INFO:yass.batch.batch:Applying function filtering...
DEBUG:yass.batch.batch:Starting parallel operation...
DEBUG:yass.batch.generator:Observations per channel: 6,000,000. Number of channels: 49. Total observations: 294,000,000 Size to traverse: 560.76 MB
DEBUG:yass.batch.generator:Max observations per batch: 26,214,363 (50 MB), 534,987 max observations per channel
DEBUG:yass.batch.generator:Number of batches: 12
DEBUG:yass.batch.generator:Last batch with 5,642,007 observations (11284014), 115,143 observations per channel
DEBUG:yass.batch.batch:Creating processes pool...
DEBUG:yass.batch.util:Processing batch 0...
DEBUG:yass.batch.util:Processing batch 1...
DEBUG:yass.batch.util:Processing batch 2...
DEBUG:yass.batch.util:Processing batch 3...
DEBUG:yass.batch.util:Processing batch 4...
DEBUG:yass.batch.util:Processing batch 5...
DEBUG:yass.batch.util:Applying function in batch 3...
DEBUG:yass.batch.util:Applying function in batch 4...
DEBUG:yass.batch.util:Applying function in batch 2.

19.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
